# Prerequisites

- Host OS: Ubuntu 20.04 lts
- Using Docker Image 'mltooling/ml-workspace-gpu' (docker pull mltooling/ml-workspace-gpu)
- Single Nvidia GPU (RTX 3080)

# Check computing resource

In [1]:
#### The number of CPU cores
!grep -c processor /proc/cpuinfo

20


In [2]:
#### GPU information
!nvidia-smi

Fri Nov 11 04:17:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 48%   51C    P8    38W / 370W |    236MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    device_count = torch.cuda.device_count()
    print("device_count: {}".format(device_count))
    for device_num in range(device_count):
        print("device {} capability {}".format(
            device_num,
            torch.cuda.get_device_capability(device_num)))
        print("device {} name {}".format(
            device_num, 
            torch.cuda.get_device_name(device_num)))
else:
    device = torch.device("cpu")
    print("no cuda device")

device_count: 1
device 0 capability (8, 6)
device 0 name NVIDIA GeForce RTX 3080


# 0. Customize Train Strategy

In [4]:
num_cpus = 16
num_gpus = 1
seed = 1234
model_name = "xlm-roberta-base" # bert-base-multilingual-cased ; klue/roberta-base ; bert-base-cased, etc.
train_proportion = 0.7 # train set : eval set = 7 : 3
do_hpo = False

# If you want to search best hyperparameters using ray tune, parameters below should be set
n_trials = 5
std = 0.1
patience = 5

# 1. Import packages

In [5]:
## Need to check if packages are compatible ##

# !pip install accelerate nvidia-ml-py3
# !pip install datasets==2.4.0
# !pip install huggingface_hub==0.9.1
# !pip install transformers==4.22.1 
# !pip install pyarrow==9.0.0
# !pip install -q ray

In [6]:
import transformers
import datasets
import huggingface_hub
import pyarrow

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(pyarrow.__version__)

# 4.22.1
# 2.4.0
# 0.9.1
# 9.0.0

4.22.1
2.4.0
0.9.1
9.0.0


In [7]:
import os
import re
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 'You can use tf32' if you are acessing Ampere hardware
import torch
torch.backends.cuda.matmul.allow_tf32 = True

from datasets import load_dataset, load_metric, ClassLabel
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.examples.pbt_transformers.utils import (
    download_data,
    build_compute_metrics_fn,
)
from ray.tune.schedulers import PopulationBasedTraining
from transformers import (
    glue_tasks_num_labels,
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)

# 2. Import Data

2 files are needed (`{data_name}_train.csv` and `{data_name}_test.csv`) in your data directory (in this case, `data_splited/`).

In [8]:
data_name = "cardiovascular_sev_dataset" 

dataset = load_dataset('csv', data_files={'train': f'../data_splited/{data_name}_train.csv',
                                          'test': f'../data_splited/{data_name}_test.csv'})
dataset

Using custom data configuration default-eb5690d969448b40
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-eb5690d969448b40/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 3756
    })
    test: Dataset({
        features: ['id', 'past_history', 'treatment_effect', 'examination', 'label'],
        num_rows: 940
    })
})

# 3. Data Preprocessing

In [9]:
#### Select the column you want to tokenize and label column.

dataset = dataset.remove_columns(['id', 'treatment_effect', 'examination'])
dataset = dataset.rename_column("past_history", "text")

# dataset = dataset.remove_columns(['id', 'examination', 'past_history'])
# dataset = dataset.rename_column("treatment_effect", "text")

# dataset = dataset.remove_columns(['id', 'treatment_effect', 'past_history'])
# dataset = dataset.rename_column("examination", "text")

print(dataset['train']['text'][0])

본 64세 남환은 HTN, DM 의 과거력 (on medication) 있는 분으로 



1주일 전부터 왼쪽 가슴쪽으로의 통증이 있어 OPD 내원하여 EKG 



상에서 Inf. Infarction 의심되고 2011.08.11 시행한 Echo 상에서 



RCA territory RWMA 있었으며 EF 51%, conc. LVH, E/E' 12  asc. 



aorta 37mm  소견 있었으며 8월 12일 Angio 위해 입원하였으나 보



호자 없어 시술하지 못하고 금일 CAG 시행위해 일일입원실 입원함


In [10]:
#### Remove NA rows

dataset = dataset.filter(lambda row: pd.notnull(row["text"]))

#### Remove specal characters

def remove_sp(example):
    example["text"]=re.sub(r'[^a-z|A-Z|0-9|ㄱ-ㅎ|ㅏ-ㅣ|가-힣| ]+', '', str(example["text"]))
    return example

dataset = dataset.map(remove_sp)

print(dataset)
print(dataset['train']['text'][0])

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-eb5690d969448b40/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-b38ea0e30a82d01f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-eb5690d969448b40/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-e00e6ae3dce802a8.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-eb5690d969448b40/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0d04eff8a8583e00.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-eb5690d969448b40/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-30e62e5b8f0cb9ea.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3756
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 940
    })
})
본 64세 남환은 HTN DM 의 과거력 on medication 있는 분으로 1주일 전부터 왼쪽 가슴쪽으로의 통증이 있어 OPD 내원하여 EKG 상에서 Inf Infarction 의심되고 20110811 시행한 Echo 상에서 RCA territory RWMA 있었으며 EF 51 conc LVH EE 12  asc aorta 37mm  소견 있었으며 8월 12일 Angio 위해 입원하였으나 보호자 없어 시술하지 못하고 금일 CAG 시행위해 일일입원실 입원함


In [11]:
#### Tokenizing 

tokenizer = AutoTokenizer.from_pretrained(model_name, truncation_side = 'left') # truncation_side = 'left' option remains last 512 tokens

def tokenize_function(examples):
    tokenized_batch = tokenizer(examples["text"], padding="max_length", truncation=True) # padding : ['longest', 'max_length', 'do_not_pad']
    return tokenized_batch

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-eb5690d969448b40/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-3aabaf0038fffc54.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-eb5690d969448b40/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-d41f9d74d055223b.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3756
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 940
    })
})

In [12]:
#### Train-Evalulation-Test Split 

train_dataset = tokenized_datasets["train"].shuffle(seed=seed).select(range(0,math.floor(len(tokenized_datasets["train"])*train_proportion)))
eval_dataset = tokenized_datasets["train"].shuffle(seed=seed).select(range(math.floor(len(tokenized_datasets["train"])*train_proportion), len(tokenized_datasets["train"])))
test_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-eb5690d969448b40/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-9372591042b600d7.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-eb5690d969448b40/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-9372591042b600d7.arrow


In [13]:
#### Applying class weights

def class_weight(train_dataset) :
    
    train_labels = np.array(train_dataset["label"])
    class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)
    
    weights = torch.tensor(class_weights, dtype = torch.float)
    
    return weights

weights = class_weight(train_dataset)
print(f"Class Weights: {weights}")

Class Weights: tensor([0.6868, 1.8385])


# 4. Set model configuration

In [14]:
#### Initialize Ray
ray.shutdown()
ray.init(log_to_driver=False, ignore_reinit_error=True, num_cpus=num_cpus, num_gpus=num_gpus, include_dashboard=False)

####  Load the model 
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        output_attentions = False,
        output_hidden_states = False
        )

#### Define metrics to use for evaluation
def compute_metrics(eval_pred):
    metric1 = load_metric("accuracy")
    metric2 = load_metric("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = metric2.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1, "objective": accuracy+f1}

#### batch size = 32, evaluate every 50 steps
training_args = TrainingArguments(
    output_dir=".",
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-5, # config
    weight_decay=0.1, # config
    adam_beta1=0.1, # config
    adam_beta2=0.1, # config
    adam_epsilon=1.5e-06, # config
    num_train_epochs=15, # config
    max_steps=-1,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,  # config
    warmup_steps=0,
    logging_dir="./logs",
    save_strategy="steps",
    no_cuda=num_gpus <= 0, 
    seed=seed,  # config
    bf16=False, # Need torch>=1.10, Ampere GPU with cuda>=11.0
    fp16=True,
    tf32=True, 
    eval_steps = 50,
    load_best_model_at_end=True,
    greater_is_better=True,
    metric_for_best_model="objective", # f1 + acc
    report_to="none",
    skip_memory_metrics=True,
    gradient_checkpointing=True
    )

#### Customize trainer class to apply class weights
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        weight = weights.to(device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    
trainer = CustomTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    )

#### Fix batch_size in each trial
tune_config = {
    "per_device_eval_batch_size": 8,
    "per_device_train_batch_size": 8,
    "max_steps": -1
}

2022-11-11 00:08:13,823	INFO worker.py:1518 -- Started a local Ray instance.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d161e8f5f6f2ed433fb4023d6cb34146506b3f/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from

# 4.5. Hyperparameter Optimization with PBT (Optional)

- If you want to train model with fixed hyperparameters in `training_args`, skip this step

In [15]:
if do_hpo == True:
    
    #### PBT schduler
    scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="objective",
        mode="max",
        perturbation_interval=1,
        hyperparam_mutations={
            "num_train_epochs": tune.randint(2, 20),
            "seed": tune.randint(1, 9999),
            "weight_decay": tune.uniform(0.0, 0.3),
            "learning_rate": tune.uniform(1e-5, 5e-5),
            "warmup_ratio": tune.uniform(0.0, 0.3),
            "adam_beta1": tune.loguniform(1e-2, 1),
            "adam_beta2": tune.loguniform(1e-3, 1),
            "adam_epsilon": tune.loguniform(1e-8, 1e-5),
        }, 
    )

    #### Define columns to report
    reporter = CLIReporter(
        parameter_columns={
            "weight_decay": "w_decay",
            "learning_rate": "lr",
            "per_device_train_batch_size": "train_bs/gpu",
            "num_train_epochs": "num_epochs",
        },
        metric_columns=["eval_f1", "eval_accuracy", "eval_objective", "eval_loss", "epoch", "training_iteration"]
    )

    #### Early stopping
    stopper = tune.stopper.ExperimentPlateauStopper(metric="objective", 
                                                    std=std,
                                                    top=n_trials,
                                                    mode="max",
                                                    patience=patience
                                                    )

    #### HPO
    hpo_result = trainer.hyperparameter_search(
        hp_space = lambda _: tune_config,
        direction = "maximize",
        backend="ray",
        reuse_actors = True,
        n_trials=n_trials,
        resources_per_trial={"cpu": num_cpus, "gpu": num_gpus},
        scheduler=scheduler,
        keep_checkpoints_num=1,
        checkpoint_score_attr="training_iteration",
        stop=stopper,
        progress_reporter=reporter,
        local_dir="./test-results",
        name="tune_transformer_pbt",
        log_to_file=True,
    )
    
    print(hpo_result)
    
    for n, v in hpo_result.hyperparameters.items():
        setattr(trainer.args, n, v)
    
    print(trainer.args)

else:
    pass

# 5. Train, evaluate the model

In [16]:
train_history = trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d161e8f5f6f2ed433fb4023d6cb34146506b3f/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d

Step,Training Loss,Validation Loss,Accuracy,F1,Objective
50,No log,0.686573,0.729370,0.000000,0.729370
100,No log,0.546336,0.797693,0.606897,1.404590
150,No log,0.361667,0.848270,0.762829,1.611099
200,No log,0.354842,0.841171,0.757781,1.598952
250,No log,0.299786,0.898846,0.821875,1.720721
300,No log,0.294433,0.898846,0.822430,1.721276
350,No log,0.293469,0.885537,0.810573,1.696110
400,No log,0.302301,0.914818,0.843648,1.758466
450,No log,0.257059,0.892635,0.822840,1.715476
500,0.445600,0.273280,0.920142,0.858044,1.778186


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1127
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1127
  Batch size = 8
/opt/conda/lib/python3.8/site-packages/transformers/trainer.py:1808: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_non

  Num examples = 1127
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1127
  Batch size = 8
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1127
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forw

In [17]:
eval_result = trainer.evaluate()
eval_result

The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1127
  Batch size = 8


{'eval_loss': 0.43250736594200134,
 'eval_accuracy': 0.9325643300798581,
 'eval_f1': 0.8754098360655737,
 'eval_objective': 1.8079741661454318,
 'eval_runtime': 8.0962,
 'eval_samples_per_second': 139.2,
 'eval_steps_per_second': 17.416,
 'epoch': 15.0}

# 6. Test results

In [18]:
pred = trainer.predict(test_dataset=test_dataset)
pred

The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 940
  Batch size = 8


PredictionOutput(predictions=array([[-3.338 ,  3.521 ],
       [ 3.303 , -2.922 ],
       [ 3.799 , -3.658 ],
       ...,
       [ 0.2651,  0.3213],
       [-2.559 ,  2.799 ],
       [-2.998 ,  2.998 ]], dtype=float16), label_ids=array([1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 

In [19]:
label_test = list(pred.label_ids)
pred_test = list(map(lambda x: x.index(max(x)), pred.predictions.tolist()))

In [20]:
print(confusion_matrix(label_test, pred_test))

[[634  29]
 [ 42 235]]


In [21]:
accuracy = accuracy_score(label_test, pred_test)
f1 = f1_score(label_test, pred_test)
recall = recall_score(label_test, pred_test)
precision = precision_score(label_test, pred_test)

print(accuracy)
print(f1)
print(recall)
print(precision)

0.924468085106383
0.8687615526802218
0.8483754512635379
0.8901515151515151


In [22]:
test_results = pd.concat([pd.DataFrame(test_dataset['text'], columns=['text']), 
                          pd.DataFrame(label_test, columns=['label']),
                          pd.DataFrame(pred_test, columns=['pred'])],
                         axis=1
                        )
test_results

,text,label,pred
0,상기 46세 남환 HTNnot med 외 특이 과거력 없는 분으로 내원 2시간 ...,1,1
1,상기 45세 남환 혈압 높다는 얘기 들었으나 medication하지 않았다고 하며 ...,0,0
2,상기 74세 남환 CAD 3VD LM sp midCAB for LAD 202004...,0,0
3,상기 55세 남환 HTN DM type 2 Dyslipidemia ESRD on P...,0,0
4,상기 남환 특이 과거력 없던 분으로 내원 3시간 50분전5AM경부터 chest pa...,1,1
...,...,...,...
935,본 환자는 특이내과적 병력 없던 분으로 1년전과 3개월전 각각 약 30분 지속되는 ...,0,0
936,상기 52세 남환 2005 HTN DM cerebral infarction Hx있는...,0,0
937,상기 57세 남환 HTN2yr DM1mo 과거력 있으며 05pack x 35yr c...,0,1
938,상기 78세 남환 고혈압no current med 당뇨on PO med 과거력 있는...,1,1


In [23]:
# test_results.to_csv('./ph_result.csv')

# 7. Save the model

In [24]:
# model_path = f"sev_exam_1.0"
# trainer.model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

In [25]:
# load model / pred

# load_model = AutoModelForSequenceClassification.from_pretrained("sev_exam_1.0/")
# load_tokenizer = AutoTokenizer.from_pretrained("sev_exam_1.0/")

# Reference

https://bo-10000.tistory.com/154  
https://huggingface.co/blog/ray-tune  
https://docs.ray.io/en/latest/tune/examples/pbt_transformers.html  
https://wood-b.github.io/post/a-novices-guide-to-hyperparameter-optimization-at-scale/#schedulers-vs-search-algorithms  
https://docs.ray.io/en/latest/tune/api_docs/search_space.html  
https://docs.ray.io/en/latest/tune/tutorials/tune-advanced-tutorial.html  
https://keras.io/examples/keras_recipes/sample_size_estimate/  
https://www.topbots.com/fine-tune-transformers-in-pytorch/  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html  
https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/  
https://docs.ray.io/en/latest/tune/faq.html  
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#population-based-training-tune-schedulers-populationbasedtraining  
https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.hyperparameter_search  
https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#optuna-tune-search-optuna-optunasearch  
https://kyunghyunlim.github.io/nlp/ml_ai/2021/09/22/hugging_face_5.html  

# Future Works
 - step이 늘어나면서 성능이 어떻게 좋아지는지, hp조합에 따라 어떻게 좋아지는지 시각화 추가